In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
#import yt
#import vtk
import h5py

%matplotlib inline

import openmc
import openmc.mgxs as mgxs

# A Simple Toy Problem

### This is a simple toy problem to practice using OpenMC. We are going to use this toy problem to help us implement an algorithm to convert group constant generated by OpenMC into a format that we can use in Moltres.

### We are going to run OpenMC on a 20% enriched uranium cylinder suspended in a cylinder of FLiBe salt.

### We start by defining the materials

In [2]:
temperatures = [922, 972, 1022, 1072, 1122]

###################
#### MATERIALS ####
###################

# Create the fuel Material
fuel_material = openmc.Material(1, "uranium_fuel")
fuel_material.add_nuclide('U235', 0.2, 'wo')
fuel_material.add_nuclide('U238', 0.8, 'wo')
fuel_material.set_density('g/cm3', 19.1) # rough approximation, good enough for this toy problem

# Create the salt Material
# Our salt is going to be a simple FLiBe salt
salt_material = openmc.Material(2, "salt_bath")
salt_material.add_nuclide('Li7', 0.1090, 'wo')
salt_material.add_nuclide('Li6', 5e-6, 'wo')
salt_material.add_nuclide('F19', 0.6680, 'wo')
salt_material.add_nuclide('Be9', 0.0627, 'wo')
fuel_material.add_element('Zr', 0.1092, 'wo')
salt_material.set_density('g/cm3', 2.146) #reference density from normal fuel salt

# Create the Materials and export to XML
materials = openmc.Materials([fuel_material, salt_material])
materials.export_to_xml()

### Next we create the geometry

In [3]:
##################
#### GEOMETRY ####
##################

#Cylinder height
H = 10.0 # [cm]

# Cylinder parameters
R_fuel = 4.0 # [cm]
R_salt = 7.0 # [cm]

# Create the cylinders
fuel_cylinder = openmc.ZCylinder(r=R_fuel)
salt_cylinder = openmc.ZCylinder(r=R_salt, boundary_type='reflective')

# Create the reactor core Region
cylinder_top_surface = openmc.ZPlane(z0=H/2, boundary_type='reflective') # Are these boundary conditions right?
cylinder_bottom_surface = openmc.ZPlane(z0=-H/2, boundary_type='reflective')
fuel_region = -fuel_cylinder & +cylinder_bottom_surface & -cylinder_top_surface
salt_region = -salt_cylinder & +fuel_cylinder & +cylinder_bottom_surface & -cylinder_top_surface # not sure if this works yet

# Create the toy Universe
toy_universe = openmc.Universe(name='toy_problem')

# Create our Cells
fuel_cell = openmc.Cell(name='fuel_cell') 
fuel_cell.fill = fuel_material
fuel_cell.region = fuel_region

salt_cell = openmc.Cell(name='salt_cell') 
salt_cell.fill = salt_material
salt_cell.region = salt_region

# Add our Cells to our Universe
toy_universe.add_cell(fuel_cell)
toy_universe.add_cell(salt_cell)

# Create the Geometry and export to XML
geometry = openmc.Geometry(toy_universe)
geometry.export_to_xml()

### Let's make sure our geometry looks correct

In [ ]:
toy_universe.plot(width =[2*R_salt, H], basis = 'xz', color_by='cell')

In [ ]:
toy_universe.plot(width = [2*R_salt, 2*R_salt], basis = 'xy', color_by='cell')

In [ ]:
###############
#### PLOTS ####
###############

# Instantiate a Plot object
top_plot = openmc.Plot(plot_id=1, name='top-plot')
side_plot = openmc.Plot(plot_id=2, name='side-plot')
vox_plot = openmc.Plot(plot_id=3, name='vox-plot')

# Plot parameters
top_plot.basis = 'xy'
side_plot.basis = 'xz'
vox_plot.type = 'voxel'

top_plot.origin = (0, 0, 0)
side_plot.origin = (0, 0, 0)

top_plot.width = (1.5*2*R_salt, 1.5*2*R_salt)
side_plot.width = (1.5*2*R_salt, 1.5*H)
vox_plot.width = (1.5*2*R_salt, 1.5*2*R_salt, 1.5*H)

top_plot.pixels = (int(10*2*R_salt), int(10*2*R_salt))
side_plot.pixels = (int(10*2*R_salt), int(10*H))
vox_plot.pixels = (int(10*2*R_salt), int(10*2*R_salt), int(10*H))

top_plot.color_by = 'material'
side_plot.color_by = 'material'
vox_plot.color_by = 'material'

# Create a Plots object and export to XML
plots = openmc.Plots([top_plot, side_plot, vox_plot])
plots.export_to_xml()

#openmc.plot()
# Plot the geometry
openmc.plot_geometry()

# Convert the generated voxel data to vtk for use in yt
#!openmc-voxel-to-vtk plot_3.h5

# Perhaps one day yt will support vtk in general
#ds = yt.load('plot.vti')

### Now we need to figure out what our settings will be

In [4]:
##################
#### SETTINGS ####
##################

# OpenMC simulation parameters
batches = 50 # number of source particle histories
inactive = 10 # number of batches where we don't cound tallies. This is to allow for convergence of the source distribution (i think)
particles = 2500 # the number of particles we simulate per batch

# Instatiate a Settings Object
settings = openmc.Settings()
settings.batches = batches
settings.inactive = inactive
settings.particles = particles

# Create an initial uniform spatial source distribution over fissionable zones
source = openmc.Source()

# Create an initial uniform spatial source distribution over fissionable zones
bounds = [-R_salt, -R_salt, -H/2, R_salt, R_salt, H/2]
uniform_dist = openmc.stats.Box(bounds[:3], bounds[3:], only_fissionable=True)
settings.source = openmc.Source(space=uniform_dist)

# Define the spatial distribution of our neutron distribtuion
# = openmc.stats.Uniform(-R_fuel,R_fuel)
#phi_dist = openmc.stats.Uniform(0,2*np.pi)
#z_dist = openmc.stats.Uniform(-H/2, H/2)
#source.space = openmc.stats.CylindricalIndependent(r_dist, phi_dist, z_dist)

# Use a simple angular distribution
#source.angle = openmc.stats.Isotropic()

# Use the defualt energy distribution by not specifying it!

# Assign our source to the settings object
#settings.source = source

# Make sure we generate output!
settings.output = {
    'tallies': True
}

# Export Settings to XML
settings.export_to_xml()

In [ ]:
#################
#### FILTERS ####
#################

# We start by making some filters
# Energy filter for multi-group cross-section Tallies
#energy_filter = openmc.EnergyFilter([0., 0.625, 20.0e6])
#cell_filter = openmc.CellFilter([fuel_cell, salt_cell])


# Create mesh which will be used for tally
mesh = openmc.RegularMesh()
mesh.dimension = [100, 100]
mesh.lower_left = [-R_salt, -R_salt]
mesh.upper_right = [R_salt, R_salt]

# Create mesh filter for tally
mesh_filter = openmc.MeshFilter(mesh)


# Create a 3D tally mesh
# I've had lots of trouble getting good looking results from a 3D tally mesh :/
#mesh = openmc.RegularMesh(mesh_id=1)
#mesh.dimension = [100, 100, 10]
#mesh.lower_left = [-R_salt, -R_salt, -H/2]
#mesh.upper_right = [R_salt, R_salt, H/2]
#meshsurface_filter = openmc.MeshSurfaceFilter(mesh)
#mesh_filter = openmc.MeshFilter(mesh)

In [5]:
#################
#### TALLIES ####
#################

# In order to measure useful quantities, we will need to use OpenMC's Tally class!
tallies_file = openmc.Tallies()

In [ ]:
scores = ['decay-rate']

# Instantiate flux Tally in moderator and fuel
flux_tally = openmc.Tally(name='flux')
flux_tally.filters = [mesh_filter]
flux_tally.scores = ['flux', 'fission']
tallies_file.append(flux_tally)

# Instantiate reaction rate Tally in fuel
fuel_rxn_tally = openmc.Tally(name='fuel rxn rates')
fuel_rxn_tally.filters = [mesh_filter]
fuel_rxn_tally.scores = ['nu-fission', 'scatter', 'kappa-fission']
fuel_rxn_tally.nuclides = ['U238', 'U235']
tallies_file.append(fuel_rxn_tally)

# Instantiate reaction rate Tally in salt
salt_rxn_tally = openmc.Tally(name='salt rxn rates')
salt_rxn_tally.filters = [mesh_filter]
salt_rxn_tally.scores = ['absorption', 'total']
salt_rxn_tally.nuclides = ['Be9']
tallies_file.append(salt_rxn_tally)

In [13]:
###########################
#### MG CROSS SECTIONS ####
###########################

# Alternatively, we can specify tallies using OpenMC's mgxs class!
# The basic strategy is to create a library object, specity the geometry, enery groups, reaction types,
# domain type, and domain itself
# build the library, and then add it to a Tallies object!

mgxs_library = mgxs.Library(geometry, by_nuclide=True)


# Instantiate a 2-group EnergyGroups object
neutron_groups = mgxs.EnergyGroups([0., 0.625, 20.0e6]) #are these right? 

# Initialize a 2-energy-group and 6-delayed-group MGXS Library
mgxs_library.energy_groups = neutron_groups
mgxs_library.num_delayed_groups = 6

# Specify multi-group cross section types to compute
mgxs_types = ['total', 'transport', 'nu-transport', 'absorption', 
              'capture', 'fission', 'nu-fission', 'kappa-fission', 
              'scatter', 'nu-scatter', 'scatter matrix', 'nu-scatter matrix', 
              'multiplicity matrix', 'nu-fission matrix', 'chi', 'chi-prompt', 
              'inverse-velocity', 'prompt-nu-fission', 'prompt-nu-fission matrix', 
              'delayed-nu-fission', 'delayed-nu-fission matrix', 'chi-delayed', 'beta']

    #['transport', 'absorption', 
    #           'fission', 'nu-fission', 'kappa-fission', 
    #          'scatter', 'scatter matrix', 
    #          'multiplicity matrix', 'chi', 
    #          'inverse-velocity', 'chi-delayed', 'beta']

    #'nu-transport', 'total', 'prompt-nu-fission', 'prompt-nu-fission matrix', 
    #'delayed-nu-fission', 'delayed-nu-fission matrix', 'nu-fission matrix',
    #'chi-prompt','nu-scatter', 'nu-scatter matrix', 'capture'

# we need absorption + scatter - in scattering, decay rate

mgxs_library.mgxs_types = mgxs_types

# Specify a "material" domain type for the cross section tally filters
mgxs_library.domain_type = 'cell'

# Specify the cell domain over which to compute multi-group cross sections
mgxs_library.domain = geometry.get_all_material_cells()#

In [14]:
# Check the library - if no errors are raised, then the library is satisfactory.
mgxs_library.check_library_for_openmc_mgxs()
# Construct all tallies needed forthe multi-group cross section library
mgxs_library.build_library()

In [8]:


# Create a "tallies.xml" file for the MGXS Library
mgxs_library.add_to_tallies_file(tallies_file, merge=True)

In [9]:
# Export to "tallies.xml"
tallies_file.export_to_xml()

/home/ooblack/miniconda3/envs/openmc-env/lib/python3.7/site-packages/openmc/mixin.py:68: IDWarning: Another Filter instance already exists with id=145.
  warn(msg, IDWarning)
/home/ooblack/miniconda3/envs/openmc-env/lib/python3.7/site-packages/openmc/mixin.py:68: IDWarning: Another Filter instance already exists with id=2.
  warn(msg, IDWarning)
/home/ooblack/miniconda3/envs/openmc-env/lib/python3.7/site-packages/openmc/mixin.py:68: IDWarning: Another Filter instance already exists with id=5.
  warn(msg, IDWarning)
/home/ooblack/miniconda3/envs/openmc-env/lib/python3.7/site-packages/openmc/mixin.py:68: IDWarning: Another Filter instance already exists with id=6.
  warn(msg, IDWarning)
/home/ooblack/miniconda3/envs/openmc-env/lib/python3.7/site-packages/openmc/mixin.py:68: IDWarning: Another Filter instance already exists with id=54.
  warn(msg, IDWarning)
/home/ooblack/miniconda3/envs/openmc-env/lib/python3.7/site-packages/openmc/mixin.py:68: IDWarning: Another Filter instance already 

In [10]:
# Run OpenMC
openmc.run()

                                %%%%%%%%%%%%%%%
                           %%%%%%%%%%%%%%%%%%%%%%%%
                        %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                      %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                   %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                                    %%%%%%%%%%%%%%%%%%%%%%%%
                                     %%%%%%%%%%%%%%%%%%%%%%%%
                 ###############      %%%%%%%%%%%%%%%%%%%%%%%%
                ##################     %%%%%%%%%%%%%%%%%%%%%%%
                ###################     %%%%%%%%%%%%%%%%%%%%%%%
                ####################     %%%%%%%%%%%%%%%%%%%%%%
                #####################     %%%%%%%%%%%%%%%%%%%%%
                ######################     %%%%%%%%%%%%%%%%%%%%
                #######################     %%%%%%%%%%%%%%%%%%
                 #######################     %%%%%%%%%%%%%%%%%
                 #####################

### Now we want to process and visualize our data!

In [11]:
#########################
#### POST PROCESSING ####
#########################

# Load the statepoint file
statepoint_file = openmc.StatePoint('statepoint.50.h5')

In [16]:
# Load our tallies as well as out MGXS library
mgxs_library.load_from_statepoint(statepoint_file)
file = mgxs_library.create_mg_library()
#file.export_to_hdf5

TypeError: Unable to set "beta" to "<openmc.mgxs.mgxs.FissionXS object at 0x7f69d9361310>" which is not of type "Beta"

In [ ]:
mgxs_library.build_hdf5_store(filename='mgxs.h5')


In [ ]:
fuel_mgxs = mgx len(np.uns_library.get_mgxs(fuel_cell, 'transport')

df = fuel_mgxs.get_pandas_dataframe()
fuel_mgxs

In [17]:
file, mat, geo = mgxs_library.create_mg_mode()

TypeError: Unable to set "beta" to "<openmc.mgxs.mgxs.FissionXS object at 0x7f69d9361310>" which is not of type "Beta"

In [ ]:
file.export_to_hdf5('mgxs-alt')

In [ ]:
# Get the OpenMC fission rate mesh tally data
flux_tally = statepoint_file.get_tally(name='flux')

In [ ]:
flux_data = flux_tally.get_slice(scores=['flux'])

In [ ]:
fission_data = flux_tally.get_slice(scores=['fission'])

In [ ]:
flux_data.mean.shape = mesh.dimension
fission_data.mean.shape = mesh.dimension
flux_data.std_dev.shape = mesh.dimension
fission_data.std_dev.shape = mesh.dimension

In [ ]:
fig = plt.subplot(121)
fig.imshow(flux_data.mean)
fig = plt.subplot(122)
fig.imshow(fission_data.mean)

In [ ]:
flux_data.mean[0,:,5]

In [ ]:
mgxs = openmc.MGXSLibrary()

In [ ]:
h5file_alt = h5py.File('mgxs-alt','r')

In [ ]:
h5file = h5py.File('mgxs.h5','r')

In [ ]:
h5file_alt['set1_U235']['294K'].keys()

In [ ]:
h5file.keys()

In [ ]:
h5file['set1_U235'].keys()

In [ ]:
h5file_alt['set1_U235']['294K']['total'][1]

In [ ]:
h5file.close()

In [ ]:
file.xsdatas

In [ ]:
!cd mgxs
!ls
testfile = h5py.File('mgxs/mgxs.h5')

In [ ]:
testfile['cell']

In [ ]:
!ls

In [ ]:
testfile.close()

In [ ]:
statepoint_file.get_tally(scores='flux')

In [ ]:
tallies_file

In [ ]:
mgxs.Library.load_from_statepoint(statepoint_file)

In [ ]:
statepoint_file.filters

In [ ]:
statepoint_file.tallies.get(149)

In [ ]:
summary_file = openmc.Summary('summary.h5')

In [ ]:
tallies_file.by geometree
lib_test = openmc.mgxs.Library(summary_file.geometry, by_nuclide=False)
lib_test.domain_type = summary_file.domain_type = 'cell'
lib_test.domain = summary_file.geometry.get_all_material_cells().values()
lib_test.mgxs_types = mgxs_types
lib_test.energy_groups = mgxs.EnergyGroups([0., 0.625, 20e6])
lib_test.num_delayed_groups = 3
lib_test.check_library_for_openmc_mgxs()
lib_test.build_library()
#tallies_test = openmc.Tallies()
#lib_test.add_to_tallies_file(tallies_test, merge=True)

In [ ]:
lib_test.load_from_statepoint(statepoint_file)

In [ ]:
lib_test.all_mgxs[1]['chi-delayed'].get_xs()[0]

In [ ]:
mgxs_library.load_from_statepoint(statepoint_file)
mgxs_library.mgxs_types

In [ ]:
statepoint_file.get_tally(scores=['chi'])

In [ ]:
statepoint_file.filters

In [ ]:
mgxs_library.load_from_statepoint(statepoint_file)

In [ ]:
egroups = []
dgroups = []
for val in statepoint_file.filters.values():
    if (isinstance(val, openmc.EnergyoutFilter)):
        egroups = val.values
    if (isinstance(val, openmc.DelayedGroupFilter)):
        dgroups = val.bins

In [ ]:
a = statepoint_file.filters.values()

In [ ]:
list(dgroups)

In [ ]:
statepoint_file.tallies